In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from torchmetrics.classification import ConfusionMatrix, ROC
from PIL import Image
import os
import matplotlib.pyplot as plt
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, ROC, ConfusionMatrix, AUROC, AveragePrecision
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve, confusion_matrix
import seaborn as sns

In [2]:
# Dataset class
from PIL import Image

class DeepfakeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.corrupted_images = []  # Store names of corrupted images
       
        # Collect all image paths
        for label, subdir in enumerate(["Real", "Fake"]):
            subdir_path = os.path.join(root_dir, subdir)
            if os.path.exists(subdir_path):
                for img_name in os.listdir(subdir_path):
                    img_path = os.path.join(subdir_path, img_name)
                    if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
                        self.image_paths.append(img_path)
                        self.labels.append(label)

        # Validate images but do not remove them
        for path in self.image_paths:
            try:
                with Image.open(path) as img:
                    img.verify()  # Check if the image is valid
            except (OSError, IOError):
                self.corrupted_images.append(path)  # Store corrupted image name

        # Print corrupted image names if found
        if self.corrupted_images:
            print("\nCorrupted Images Detected:")
            for img_path in self.corrupted_images:
                print(img_path)
        else:
            print("\nNo corrupted images found.")

    def __len__(self):
        return len(self.image_paths)
   
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        try:
            image = Image.open(image_path).convert("RGB")
        except (OSError, IOError):
            print(f"Warning: Unable to open {image_path}. Returning blank image.")
            image = Image.new("RGB", (512, 512), (0, 0, 0))  # Return a black image
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)


In [3]:
# Preprocessing
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = DeepfakeDataset(root_dir="/home/libra-03/Desktop/DL_Assignment/new_sample_1.8L/Train", transform=transform)
val_dataset = DeepfakeDataset(root_dir="/home/libra-03/Desktop/DL_Assignment/new_sample_1.8L/Val", transform=transform)
test_dataset = DeepfakeDataset(root_dir="/home/libra-03/Desktop/DL_Assignment/new_sample_1.8L/Test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)



No corrupted images found.

No corrupted images found.

No corrupted images found.


In [4]:
# Model
class DeepfakeDetector(nn.Module):
    def __init__(self):
        super(DeepfakeDetector, self).__init__()
        self.model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.IMAGENET1K_V1)
        self.model.classifier = nn.Sequential(
            nn.Linear(self.model.classifier[1].in_features, 2048),
            nn.ReLU(),
            nn.Dropout(0.8),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(0.8),
            nn.Linear(2048, 1),
            nn.Sigmoid()
        )
   
    def forward(self, x):
        return self.model(x)

In [5]:
# Training Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using GPU")
model = DeepfakeDetector().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.0001)

Using GPU


In [6]:

# Training Loop
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
       
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")
       
        # Validation
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device).unsqueeze(1)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
       
        print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


In [ ]:
# Train
train_model(model, train_loader, val_loader, epochs=20)

Epoch 1, Loss: 0.2022
Validation Loss: 0.1286, Accuracy: 95.34%
Epoch 2, Loss: 0.1401
Validation Loss: 0.1115, Accuracy: 95.79%
Epoch 3, Loss: 0.1228
Validation Loss: 0.0955, Accuracy: 96.41%
Epoch 4, Loss: 0.1127
Validation Loss: 0.0897, Accuracy: 96.58%
Epoch 5, Loss: 0.1049
Validation Loss: 0.0872, Accuracy: 96.62%
Epoch 6, Loss: 0.0992
Validation Loss: 0.0802, Accuracy: 96.83%


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)
            outputs = model(images)
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    print(f"Test Accuracy: {100 * correct / total:.2f}%")
    
    # Generate Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Real", "Fake"], yticklabels=["Real", "Fake"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()
    
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    
    # False Positive Rate (FPR) & False Negative Rate (FNR)
    fp = cm[0][1]
    fn = cm[1][0]
    tn = cm[0][0]
    tp = cm[1][1]
    fpr = fp / (fp + tn)
    fnr = fn / (fn + tp)
    print(f"False Positive Rate (FPR): {fpr:.2f}")
    print(f"False Negative Rate (FNR): {fnr:.2f}")

# Call the function
evaluate_model(model, test_loader)


In [ ]:
#Save Model
torch.save(model.state_dict(), "Sample_new_1.8L_20_epochs_deepfake_detector_linkedin.pth")
print("Model saved successfully!")

In [ ]:
import os
print("Model will be saved in:", os.getcwd())